In [33]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import lightgbm as lgb

### https://www.kaggle.com/datasets/samybaladram/palmers-penguin-dataset-extended

In [15]:
data = pd.read_csv('/kaggle/input/palmers-penguin-dataset-extended/palmerpenguins_extended.csv')

In [23]:
data

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,health_metrics,year,species_Adelie,species_Chinstrap,species_Gentoo,island_Biscoe,island_Dream,island_Torgensen,diet_fish,diet_krill,diet_parental,diet_squid,life_stage_adult,life_stage_chick,life_stage_juvenile
0,53.4,17.8,219.0,5687.0,0,0,2021,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,49.3,18.1,245.0,6811.0,0,0,2021,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,55.7,16.6,226.0,5388.0,0,0,2021,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,38.0,15.6,221.0,6262.0,0,0,2021,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,60.7,17.9,177.0,4811.0,0,0,2021,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3425,44.0,20.4,252.0,6447.0,0,1,2025,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3426,54.5,25.2,245.0,6872.0,0,1,2025,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3427,51.4,20.4,258.0,7409.0,0,0,2025,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3428,55.9,20.5,247.0,6491.0,0,1,2025,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [17]:
data.isnull().sum()

species              0
island               0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
sex                  0
diet                 0
life_stage           0
health_metrics       0
year                 0
dtype: int64

In [21]:
data['sex'] = data['sex'].apply(lambda x: 1 if x == 'Male' else 0)
data['health_metrics'] = data['health_metrics'].apply(lambda x: 1 if x=='healthy' else 0)
numinal_cols ={col : data[col].unique() for col in data.columns if data[col].dtypes =='object'}

In [22]:
for col in numinal_cols :
    dummie = pd.get_dummies(data[col],prefix=col).astype(np.float)
    data = pd.concat([data,dummie],axis=1)
    data = data.drop(col,axis=1)

/tmp/ipykernel_32/3443676411.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dummie = pd.get_dummies(data[col],prefix=col).astype(np.float)
/tmp/ipykernel_32/3443676411.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dummie = pd.get_dummies(data[col],prefix=col).astype(np.float)
/tmp/ipykernel_32/3443676411.py:2: DeprecationWarnin

In [25]:
y = data['health_metrics']
x = data.drop('health_metrics',axis=1)

In [26]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [29]:
x_train , x_test , y_train , y_test = train_test_split(x,y,train_size=0.745,random_state=13)

In [46]:
model = lgb.LGBMClassifier(n_estimators=100,num_leaves=41,random_state=13)
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.8651428571428571